<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Diferencias-con-respecto-al-modelo-1" data-toc-modified-id="Diferencias-con-respecto-al-modelo-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Diferencias con respecto al modelo 1</a></span></li><li><span><a href="#FUNCIONES" data-toc-modified-id="FUNCIONES-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>FUNCIONES</a></span></li><li><span><a href="#TRAIN" data-toc-modified-id="TRAIN-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>TRAIN</a></span></li><li><span><a href="#TEST" data-toc-modified-id="TEST-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>TEST</a></span></li><li><span><a href="#Usando-pipelines" data-toc-modified-id="Usando-pipelines-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Usando pipelines</a></span><ul class="toc-item"><li><span><a href="#TEST-PIPELINE" data-toc-modified-id="TEST-PIPELINE-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>TEST PIPELINE</a></span></li></ul></li></ul></div>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import RobustScaler

#Automcompletar rápido
%config IPCompleter.greedy=True

from xgboost import XGBRegressor


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

C:\Users\acrespod\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\acrespod\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Diferencias con respecto al modelo 1
He usado informacion sobre:  
   - genero de la pelicula  
   -  los paises de produccion  
   - las compañias de produccion    
   - keywords de la peli  
   - nombres del cast  
   - nombres comunes de crew    
Todas estas variables han sido codificadas con la tecnica one hot encoding.    


Después cada parte pongo las variables de cada uno

**df_prod_country**
    'production_countries_United States of America'

**df_prod_companies**
     'production_companies_Paramount Pictures',

**df_Keywords**
    'Keywords_freq_rs',
 'Keywords_independent film',
 'Keywords_murder'



He extraido informacion mediante el resumen breve de la sinopsis de la pelicula (overview).Para ello, he usado una metricas tfidf

En la ultima seccion se ha creado el codigo ejecucion con sklearn pipelines

In [3]:
variables_finales = ['budget_rs',
 'popularity_rs',
 'rd_year_rs',
 'runtime_rs',
 'crew_freq_rs',
 'rd_mes_rs',
 'cast_genero_1_rs',
 'cast_genero_2_rs',
 'man',
 'rd_dia_semana_rs',
 'Keywords_freq_rs',
 'cast_genero_0_rs',
 'secret',
 'family',
 'time',
 'police',
 'Keywords_independent film',
 'woman',
 'small',
 'Keywords_murder',
 'life',
 'friend',
 'production_companies_Paramount Pictures',
 'war',
 'years',
 'friends',
 'lives',
 'love',
 'mother',
 'production_countries_United States of America',
 'day',
 'aos',
 'team',
 'boy',
 'school',
 'finds']

# FUNCIONES

In [4]:
def frecuencias_series(series, lista_posible, grafico=False):
    """series: serie con elementos en string separados por comas
        lista_posible: todas las posibles strings en series
        """
    aux = pd.Series( dtype='float64')
    for elemento in list(lista_posible):
    
        n = series.apply(lambda x:  elemento in x).sum()/2100*100
        aux = pd.concat([aux,pd.Series({elemento:n})] )
    if grafico:
        aux.sort_values().plot.barh()
    return aux.sort_values()


def lista_posibles(series):
    aux = series.map(lambda x: sorted([d['name'] for d in get_dictionary(x)]))
    generos = set()
    for f in aux:
        generos.update(f)
    return generos



def get_dictionary(s):
    try:
        d = eval(s)
    except:
        d = {}
    return d

def get_temp_var(df_cat):
    a = df_cat.release_date.dt.year
    b = df_cat.release_date.dt.month
    c = df_cat.release_date.dt.dayofweek
    d = df_cat.release_date.dt.quarter
    x = pd.concat([a,b,c,d],axis=1)
    x.columns = ['rd_year', 'rd_mes', 'rd_dia_semana', 'rd_cuatrimestre']
    return x


def conteo_genero(serie):
    aux = []
    for numero in [0,1,2]:
        x = serie.apply(lambda lista: lista.count(numero))
        aux.append(x)
    df_aux = pd.concat(aux,axis=1)
    df_aux.columns = ['cast_genero_0','cast_genero_1','cast_genero_2']
    return df_aux



In [5]:
def procesamiento(cat):
    cat = cat.copy()
    #ELIMINAR
    a_eliminar = ['belongs_to_collection','homepage','imdb_id','poster_path','tagline','spoken_languages',]
    cat.drop(columns=a_eliminar,inplace=True)

    #IMPUTACION
    cat.genres.fillna('OTRO', inplace=True)
    cat.production_companies.fillna('OTRO',inplace=True)
    cat.production_countries.fillna('OTRO',inplace=True)
    cat.Keywords.fillna('OTRO',inplace=True)
    cat.cast.fillna('OTRO',inplace=True)
    cat.crew.fillna('OTRO',inplace=True)

    cat.release_date = pd.to_datetime(cat.release_date)



    # LISTA VARIABLES
    lista_vars =['genres','production_companies','production_countries','Keywords','cast','crew']
    no_transformadas = [v for v in cat.columns.tolist() if v not in lista_vars]
    no_transformadas

    #DF VARIABLES SIN JSONS
    aux = []
    for var in lista_vars:
        z = cat[var].map(lambda x: sorted([d['name'] for d in get_dictionary(x)])).map(lambda x: ','.join(map(str, x)))
        aux.append(z)


    df_limpios = pd.concat(aux,axis=1)



    #DF FRECUENCIAS DE VARIABLES LIMPIAS
    aux = []
    for column in df_limpios.columns:
        z = df_limpios[column].map(lambda x: len(x.split(',')))
        aux.append(z)
    df_freq = pd.concat(aux,axis=1)
    df_freq.columns = [var+"_freq" for var in df_freq.columns]
    df_freq

    # VARIABLES TIEMPO
    df_temp = get_temp_var(cat)
    df_temp['rd_year'] = np.where(df_temp['rd_year']>2020,df_temp['rd_year']-100,df_temp['rd_year'])

    # frecuencia genero
    z = cat['cast'].map(lambda x: sorted([d['gender'] for d in get_dictionary(x)]))
    df_cast_genero_freq = conteo_genero(z)    
    
    
    # UNIR TODOS LOS DATASETS
    df_cat = pd.concat([cat[no_transformadas],df_temp,df_limpios,df_freq,df_cast_genero_freq],axis=1)


    # VARIABLES NUEVA WINSORIZADO
    df_cat['bin_english'] = np.where(df_cat.original_language=='en',1,0) 
    df_cat['genres_freq'] = np.where(df_cat['genres_freq']>4,4,df_cat['genres_freq'])
    df_cat['production_companies_freq'] = np.where(df_cat['production_companies_freq']>4,4,df_cat['production_companies_freq'])
    df_cat['production_countries_freq'] = np.where(df_cat['production_countries_freq']>1,1,0)


    variable = 'Keywords_freq'
    minimo = 0
    maximo = 35
    df_cat[variable].clip(minimo,maximo, inplace=True)

    variable = 'cast_freq'
    minimo = 0
    maximo = 85
    df_cat[variable].clip(minimo,maximo, inplace=True)

    variable = 'crew_freq'
    minimo = 0
    maximo = 135
    df_cat[variable].clip(minimo,maximo, inplace=True)
    return df_cat

In [6]:
# Feature engineer 2
# Funciones Ohe hot encoding

def top_ohe(series, numero_top = 10):
    """devuelve variable binarias como estan
    series: serie
    """
    
    aux = []
    
    # Top items
    top_items = show_top_items(series,numero_top=10)
    
    for g in top_items:
        x = series.apply(lambda x: 1  if  g == x else 0 )
        x.name = series.name +"_"+g
        aux.append(x)
    
    return pd.concat(aux,axis=1)


def top_ohe_uniques(series, numero_top = 10):
    """devuelve variable binarias de valores unicos
    series: serie
    """
    
    aux = []
    
    # Top items
    top_items = show_top_items_unique(series,numero_top=10)
    
    for g in top_items:
        x = series.apply(lambda x: 1  if  g in x else 0 )
        x.name = series.name +"_"+g
        aux.append(x)
    
    return pd.concat(aux,axis=1)

########################################################
# estas funciones se usan como INPUTS en las funciones de arriba

def show_top_items(series,numero_top=10, freq=False):
    """ devuelven los datos como estan (concatenados)"""
    
    mas_comunes = Counter(series).most_common(numero_top+10)
    top_items = [m[0] for m in mas_comunes]
    
    if freq:
        return mas_comunes
    
    if "" in top_items:
        top_items.remove("")
    top_items = top_items[:numero_top]
    top_items = [i.replace(',','_') for i in top_items ]
    return top_items

def show_top_items_unique(series,numero_top=10, freq=False):
    """ devuelven los elementos unicos mas frecuentes"""
    
    #mas comunes
    mas_comunes = Counter([i for j in series for i in j.split(',')]).most_common(10+numero_top)
    
    #cogemos los el indice
    top_items = [m[0] for m in mas_comunes]
    
    if freq:
        return mas_comunes
    
    if "" in top_items:
        top_items.remove("")
    top_items = top_items[:numero_top]
    top_items = [i.replace(',','_') for i in top_items ]
    return top_items

In [7]:
def get_ohe_var_series(series,elemento):
    """series : series
        elemento :str
    """
    
    series_ohe = series.apply(lambda x: 1  if  elemento in x else 0 )
    series_ohe.name = series.name +"_"+elemento
    return series_ohe
    

In [8]:
def post_procesamiento_m1(_df_cat):
    
    diccionario_variables = {
        'production_countries':['United States of America'],
        'production_companies'  :['Paramount Pictures'],
        'Keywords':['freq_rs', 'independent film', 'murder']
        
    }

    aux = []
    for variable, terminos in diccionario_variables.items():

        for termino in terminos:
            z= get_ohe_var_series( _df_cat[variable],termino)
            aux.append(z)
            
    df_fei2 = pd.concat(aux,axis=1)
    df_cat2 = pd.concat([_df_cat,df_fei2],axis=1)


    return df_cat2

In [9]:
def pre_procesamiento(df):
    df = df.copy()
    
    mean = df.runtime.mean()
    df.runtime.fillna(mean,inplace=True)
    
    variable = 'runtime'
    minimo = 70
    maximo = 200

    df[variable].clip(minimo,maximo, inplace=True)

    variable = 'budget'
    minimo = 0
    maximo = 200000000

    df[variable].clip(minimo,maximo, inplace=True)

    variable = 'popularity'
    minimo = 0
    maximo = 50

    df[variable].clip(minimo,maximo, inplace=True)
    
    return df

# TRAIN

In [10]:
# DATOS ENTRENAMIENTO
ruta_proyecto =  'C:/proyectos/03_PREDICCION_TAQUILLA'


In [11]:
# datos train
nombre_fichero_datos = 'train.csv'
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

datos_train = pd.read_csv(ruta_completa)


In [19]:
cat = pre_procesamiento(datos)

In [30]:
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                3000 non-null   float64
 15  spok

In [53]:
df_cat =  procesamiento(cat)

In [54]:
df_cat2 = post_procesamiento_m1(df_cat)
df_cat2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 36 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   id                                             3000 non-null   int64         
 1   budget                                         3000 non-null   int64         
 2   original_language                              3000 non-null   object        
 3   original_title                                 3000 non-null   object        
 4   overview                                       2992 non-null   object        
 5   popularity                                     3000 non-null   float64       
 6   release_date                                   3000 non-null   datetime64[ns]
 7   runtime                                        3000 non-null   float64       
 8   status                                         3000 non-nu

In [46]:
m2b = XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, enable_categorical=False,
                              gamma=0.5, gpu_id=-1, importance_type=None,
                              interaction_constraints='', learning_rate=0.01,
                              max_delta_step=0, max_depth=3, min_child_weight=1,
                               monotone_constraints='()',
                              n_estimators=500, n_jobs=8, num_parallel_tree=1,
                              predictor='auto', random_state=0, reg_alpha=0,
                              reg_lambda=0.01, scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None)

In [15]:
stop = pd.read_csv(ruta_proyecto + '/01_Documentos/stop_words_english.txt',names = ['termino'])

def quitar_tildes(palabra):
    #Definimos la versión con tildes y símbolos y la sin
    con = 'áéíóúüñÁÉÍÓÚÜÑ'
    sin = 'aeiouunAEIOUUN'
    #Creamos un traductor
    traductor = str.maketrans(con,sin)
    #Aplicamos el traductor y devolvemos la palabra limpia
    return(palabra.translate(traductor))

stop['limpias'] = stop.termino.transform(quitar_tildes)
def get_variables_tfidf(stop, df_cat2):
    # #Instanciamos
    tfidf = TfidfVectorizer(strip_accents = 'unicode',
                     stop_words = stop.limpias.to_list(),
                     max_df = 0.7,
                     min_df = 50,
                     ngram_range = (1,3),
                     max_features = 50)

    # #Entrenamos y aplicamos
    df_tfidf_i = tfidf.fit_transform(df_cat2.overview.fillna(''))
    df_tfidf = pd.DataFrame.sparse.from_spmatrix(df_tfidf_i,columns = tfidf.get_feature_names(),index=df_cat2.id)

    palabras_elegidas = ['man',
 'secret',
 'family',
 'time',
 'police',
 'woman',
 'small',
 'life',
 'friend',
 'war',
 'years',
 'friends',
 'lives',
 'love',
 'mother',
 'day',
 'aos',
 'team',
 'boy',
 'school',
 'finds']
    palabras_que_no_estan =[column for column in palabras_elegidas if column not in df_tfidf.columns]
    for palabra in palabras_que_no_estan:
        df_tfidf[palabra]=0
        
    return df_tfidf.loc[:,palabras_elegidas]


df_tfidf =  get_variables_tfidf(stop, df_cat2)


C:\Users\acrespod\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'daren', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'itse', 'll', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [67]:
variables_proyecto =['budget',
 'popularity',
 'rd_year',
 'runtime',
 'crew_freq',
 'rd_mes',
 'cast_genero_1',
 'cast_genero_2',
 'rd_dia_semana',
 'cast_genero_0',
 'Keywords_freq',
 'Keywords_independent film',
 'Keywords_murder',
 'production_companies_Paramount Pictures',
 'production_countries_United States of America']

df_cat3 = pd.concat([df_cat2[variables_proyecto],df_tfidf[palabras_elegidas].reset_index() ],axis=1).set_index('id').copy()


In [72]:
# TRAIN / TEST
y = df_cat2['revenue'].transform(np.log1p)
x= df_cat3.copy()


In [73]:
# ROBUST SCALER
var_rs = ['budget','popularity','rd_year','runtime',
         'crew_freq','rd_mes','cast_genero_1','cast_genero_2',
         'rd_dia_semana','cast_genero_0']
rs = RobustScaler(quantile_range=(10.0, 90.0))

In [74]:
# PIPELINE
ct = make_column_transformer(
    (rs, var_rs),
    remainder='passthrough')

# ct.fit_transform(x)

In [75]:
pipe_entrenamiento = make_pipeline(ct,m2b)

In [76]:
pipe_entrenamiento.fit(x,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('robustscaler',
                                                  RobustScaler(quantile_range=(10.0,
                                                                               90.0)),
                                                  ['budget', 'popularity',
                                                   'rd_year', 'runtime',
                                                   'crew_freq', 'rd_mes',
                                                   'cast_genero_1',
                                                   'cast_genero_2',
                                                   'rd_dia_semana',
                                                   'cast_genero_0'])])),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsam...
                            

# TEST

In [80]:
nombre_fichero_datos = "test.csv"
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

datos = pd.read_csv(ruta_completa)


In [91]:
cat = pre_procesamiento(datos)
df_cat =  procesamiento(cat)
df_cat2 = post_procesamiento_m1(df_cat)
df_tfidf  =  get_variables_tfidf(stop, df_cat2)


variables_proyecto =['budget',
 'popularity',
 'rd_year',
 'runtime',
 'crew_freq',
 'rd_mes',
 'cast_genero_1',
 'cast_genero_2',
 'rd_dia_semana',
 'cast_genero_0',
 'Keywords_freq',
 'Keywords_independent film',
 'Keywords_murder',
 'production_companies_Paramount Pictures',
 'production_countries_United States of America']

df_cat3 = pd.concat([df_cat2[variables_proyecto],df_tfidf[palabras_elegidas].reset_index() ],axis=1).set_index('id').copy()


C:\Users\acrespod\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'daren', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'itse', 'll', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [143]:
len(variables_proyecto)

15

In [99]:
# df_cat3.info()
df_cat3.reset_index

,budget,popularity,rd_year,runtime,crew_freq,rd_mes,cast_genero_1,cast_genero_2,rd_dia_semana,cast_genero_0,...,friends,lives,love,mother,day,aos,team,boy,school,finds
id,,,,,,,,,,,,,,,,,,,,,
3001,0,3.851534,2007.0,90.0,2,7.0,3,0,5.0,4,...,0.367590,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
3002,88000,3.559789,1958.0,70.0,8,5.0,2,2,6.0,6,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
3003,0,8.085194,1997.0,100.0,10,5.0,4,5,4.0,0,...,0.000000,0.000000,0.398307,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
3004,6800000,8.596012,2010.0,130.0,69,9.0,3,5,5.0,15,...,0.000000,0.000000,0.553859,0.658021,0.0,0.0,0.0,0.0,0.000000,0.0
3005,2000000,3.217680,2005.0,92.0,14,2.0,0,4,4.0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7394,42000000,9.970359,2001.0,118.0,11,8.0,4,6,4.0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
7395,19000000,6.046516,2004.0,95.0,94,8.0,5,9,4.0,8,...,0.387658,0.396817,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
7396,16000000,9.596883,1982.0,129.0,12,12.0,4,13,2.0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0


In [104]:
def submit(_df):
    # la prediccion el modelo la hace log revenue. hay que cambiar
    prediction = np.exp(pipe_entrenamiento.predict(_df))
    df_submit = pd.DataFrame({'revenue':prediction}, index=_df.reset_index().id)
    nombre_cat = "submit.csv"
    df_submit.to_csv(ruta_proyecto + '/03_Notebooks/03_Sistema/' + nombre_cat)
    
submit(df_cat3)

# Usando pipelines

In [13]:
stop = pd.read_csv(ruta_proyecto + '/01_Documentos/stop_words_english.txt',names = ['termino'])

def quitar_tildes(palabra):
    #Definimos la versión con tildes y símbolos y la sin
    con = 'áéíóúüñÁÉÍÓÚÜÑ'
    sin = 'aeiouunAEIOUUN'
    #Creamos un traductor
    traductor = str.maketrans(con,sin)
    #Aplicamos el traductor y devolvemos la palabra limpia
    return(palabra.translate(traductor))

stop['limpias'] = stop.termino.transform(quitar_tildes)

In [14]:
from sklearn.preprocessing import FunctionTransformer

cat = pre_procesamiento(datos_train)
df_cat =  procesamiento(cat)
df_cat2 = post_procesamiento_m1(df_cat)
df_cat2.overview = df_cat2.overview.fillna('')

y = np.log1p(df_cat2['revenue'])
x = df_cat2.drop(columns=['revenue'])



var_reescalado_robusto = ['budget','popularity','rd_year','runtime',
         'crew_freq','rd_mes','cast_genero_1','cast_genero_2',
         'rd_dia_semana','cast_genero_0']
reescalado_robusto = RobustScaler(quantile_range=(10.0, 90.0))

tfidf = TfidfVectorizer(strip_accents = 'unicode',
                     stop_words = stop.limpias.to_list(),
                     max_df = 0.7,
                     ngram_range = (1,3),
                     max_features = 50)

m2b = XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, enable_categorical=False,
                              gamma=0.5, gpu_id=-1, importance_type=None,
                              interaction_constraints='', learning_rate=0.01,
                              max_delta_step=0, max_depth=3, min_child_weight=1,
                               monotone_constraints='()',
                              n_estimators=500, n_jobs=8, num_parallel_tree=1,
                              predictor='auto', random_state=0, reg_alpha=0,
                              reg_lambda=0.01, scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None)


In [17]:
# FUNCIONES

def seleccionar_variables_tdidf(df_tfidf_i):
    df_tfidf = pd.DataFrame.sparse.from_spmatrix(df_tfidf_i,columns = tfidf.get_feature_names())

    palabras_elegidas = ['man',
 'secret',
 'family',
 'time',
 'police',
 'woman',
 'small',
 'life',
 'friend',
 'war',
 'years',
 'friends',
 'lives',
 'love',
 'mother',
 'day',
 'aos',
 'team',
 'boy',
 'school',
 'finds']
    palabras_que_no_estan =[column for column in palabras_elegidas if column not in df_tfidf.columns]
    for palabra in palabras_que_no_estan:
        df_tfidf[palabra]=0
    return df_tfidf[palabras_elegidas].sparse.to_dense()

def selector_variables(df):
    return df

# FUNCIONES A TRANSFORMERS
# seleccionar_variables_tdidf_pipe=FunctionTransformer(seleccionar_variables_tdidf,kw_args={'tfidf':[tfidf]})
seleccionar_variables_tdidf_pipe=FunctionTransformer(seleccionar_variables_tdidf)
selector_variables_pipe=FunctionTransformer(selector_variables)

#PIPELINE TEXTO
text_features_pipe = make_pipeline(tfidf,seleccionar_variables_tdidf_pipe)

text_features_pipe.fit_transform(x['overview'])
text_features_pipe.named_steps.tfidfvectorizer.get_feature_names()


variables_ohe=[ 'Keywords_freq',
     'Keywords_independent film',
     'Keywords_murder',
     'production_companies_Paramount Pictures',
     'production_countries_United States of America']

# COLUMN TRANSFORM
column_transformer_total = make_column_transformer(
    (reescalado_robusto, var_reescalado_robusto),
    (text_features_pipe,'overview'),
    (selector_variables_pipe,variables_ohe),
    remainder='drop')


# variables tfidf
palabras_elegidas = ['man',
 'secret',
 'family',
 'time',
 'police',
 'woman',
 'small',
 'life',
 'friend',
 'war',
 'years',
 'friends',
 'lives',
 'love',
 'mother',
 'day',
 'aos',
 'team',
 'boy',
 'school',
 'finds']


variables = var_reescalado_robusto + palabras_elegidas +variables_ohe

# PIPELINE
pipe_entrenamiento = make_pipeline(column_transformer_total,m2b)

In [19]:
pipe_entrenamiento.fit(x,y)

C:\Users\acrespod\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'daren', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'itse', 'll', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('robustscaler',
                                                  RobustScaler(quantile_range=(10.0,
                                                                               90.0)),
                                                  ['budget', 'popularity',
                                                   'rd_year', 'runtime',
                                                   'crew_freq', 'rd_mes',
                                                   'cast_genero_1',
                                                   'cast_genero_2',
                                                   'rd_dia_semana',
                                                   'cast_genero_0']),
                                                 ('pipeline',
                                                  Pipeline(steps=[('tfidfvectorizer',
                                                                   TfidfVectoriz

In [20]:
# ENTRENAMIENTO
pipe_entrenamiento.predict(x)

array([16.695292, 18.017952, 16.653389, ..., 18.584694, 18.316553,
       17.929972], dtype=float32)

In [24]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 35 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   id                                             3000 non-null   int64         
 1   budget                                         3000 non-null   int64         
 2   original_language                              3000 non-null   object        
 3   original_title                                 3000 non-null   object        
 4   overview                                       3000 non-null   object        
 5   popularity                                     3000 non-null   float64       
 6   release_date                                   3000 non-null   datetime64[ns]
 7   runtime                                        3000 non-null   float64       
 8   status                                         3000 non-nu

## TEST PIPELINE

In [26]:

nombre_fichero_datos = "test.csv"
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos
datos = pd.read_csv(ruta_completa)

cat = pre_procesamiento(datos)
df_cat =  procesamiento(cat)
df_cat2 = post_procesamiento_m1(df_cat)
df_cat2['overview'] = df_cat2['overview'].fillna('')

In [27]:
predicciones = np.expm1(pipe_entrenamiento.predict(df_cat2))
predicciones

array([  448480.3,  1187293.1,  4515844.5, ..., 42522752. ,  3538798.2,
        2510489.2], dtype=float32)